In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc=spark.sparkContext

# 1. Cree un RDD importes a partir de los datos adjuntos a esta lección como recurso. Emplee acumuladores para obtener el total de ventas realizadas y el importe total de las ventas.

In [20]:
rdd_importes = sc.textFile('./data/rdd.txt')
acum_total_ventas = sc.accumulator(0)
acum_cant_ventas = sc.accumulator(0)

In [21]:
rdd_importes.foreach(lambda x: acum_cant_ventas.add(1))
acum_cant_ventas.value

10000

In [23]:
rdd_importes.count()

10000

In [22]:
rdd_importes.foreach(lambda x: acum_total_ventas.add(float(x)))
acum_total_ventas.value

5042335.0

# 2. Si se conoce que a cada venta hay que restarle un importe fijo igual a 10 pesos por temas de impuestos.

### a. ¿Cómo restaría este impuesto de cada venta utilizando una variable broadcast para acelerar el proceso?

In [36]:
br_imp = sc.broadcast(10)
acum_ventas_sin_imp = sc.accumulator(0)

In [37]:
ventas_sin_impuestos = rdd_importes.map(lambda x: float(x) - br_imp.value)
ventas_sin_impuestos.take(5)

[517.0, 376.0, 691.0, 230.0, 931.0]

### b. Cree un RDD llamado ventas_sin_impuestos a partir de la propuesta del inciso a que contenga las ventas sin impuestos.

In [32]:
rdd_importes.foreach(lambda x: acum_ventas_sin_imp.add(float(x) - br_imp.value))
acum_ventas_sin_imp.value

4942335.0

### c. Destruya la variable broadcast creada luego de emplearla para crear el RDD del inciso b.

In [38]:
br_imp.destroy()

# 2. Persista el RDD ventas_sin_impuestos en los siguientes niveles de persistencia.
        * Memoria
        * Disco solamente 
        * Memoria y disco

In [34]:
from pyspark.storagelevel import StorageLevel

In [39]:
ventas_sin_impuestos.cache()

PythonRDD[30] at RDD at PythonRDD.scala:53

In [42]:
ventas_sin_impuestos.unpersist()

PythonRDD[30] at RDD at PythonRDD.scala:53

In [41]:
ventas_sin_impuestos.persist(StorageLevel.DISK_ONLY)

PythonRDD[30] at RDD at PythonRDD.scala:53

In [43]:
ventas_sin_impuestos.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[30] at RDD at PythonRDD.scala:53